# <div style="text-align: center;">Cifrado con curvas elípticas</div>

<div style="text-align: justify;">
    El cifrado asimétrico es una técnica criptográfica que utiliza pares de claves distintas para encriptar y desencriptar datos. Cada par de claves consiste en una clave pública, que puede ser compartida, y una clave privada, que debe mantenerse en secreto. Esto permite a cualquier persona cifrar datos utilizando la clave pública del destinatario, pero solo el destinatario a través de su clave privada podrá descifrar la información.
    El cifrado con curvas elípticas es una forma de cifrado asimétrico que emplea las curvas elípticas para realizar operaciones criptográficas. Este método ofrece ventajas en términos de eficiencia y seguridad en comparación con otros métodos asimétricos, como RSA, ya que utiliza propiedades de las curvas elípticas para la generación de claves seguras.

El sencillo programa en la parte inferior implementa un sistema básico de cifrado utilizando curvas elípticas (ECC) y una función XOR:
</div>
<div style="text-align: justify;">
Primero, se genera una clave privada aleatoria a partir de un número aleatorio comprendido entre 0 y el orden de la curva elíptica `secp192r1`. Una vez generada la clave privada, se calcula su correspondiente clave pública. Luego, se definen las funciones para cifrar (enkryptar) y descifrar (desenkryptar) mensajes utilizando XOR y la clave privada correspondiente.
    La función `xor_simple(data, clave)` utiliza XOR (or exclusivo) para combinar cada byte del mensaje original con la clave generada dinámicamente. Esto permite que dicho mensaje solo pueda descifrarse utilizando una clave "compartida".

</div>

<div style="text-align: center;">
  <img src="https://miro.medium.com/v2/resize:fit:1400/1*_wRwisFsygokN6YQTIEq2Q.png" alt="Elliptic Curve">
</div>


<div style="text-align: justify;">
    La criptografía de curvas elípticas (ECC) no proporciona directamente un método de cifrado. En su lugar, se suele diseñar un esquema de cifrado híbrido utilizando el esquema de intercambio de claves ECDH (Diffie-Hellman) para derivar una clave secreta compartida para el cifrado y descifrado simétrico de datos.
</div>

# <div style="text-align: center;">Problema del Logaritmo Discreto </div>

<div style="text-align: justify;">
    Las curvas elípticas proporcionan un alto nivel de seguridad con claves más pequeñas comparadas con otros métodos, como RSA. La seguridad de estos sistemas de cifrado se basa en la dificultad del Problema del Logaritmo Discreto en el contexto de las curvas elípticas.
    El problema del logaritmo discreto en curvas elípticas se define de la siguiente manera: dado un punto P en una curva elíptica y un punto Q = k * P (donde k es un entero desconocido), encontrar k es extremadamente difícil. 
Por ejemplo, un campo finito de 256 bits implica que el número total de puntos en la curva elíptica es de aproximadamente 2^256, lo que hace que este problema sea computacionalmente intratable con los algoritmos actuales.
</div>

In [3]:
pip install tinyec

Note: you may need to restart the kernel to use updated packages.


In [2]:
import secrets as sc
from tinyec import registry
from tkinter import *
from tkinter import ttk
import base64
import binascii
CURVA = registry.get_curve('secp192r1')
def regenerate_keys():
    global keys, claveKompartida, privClave, pubClave
    keys = generate_keys()
    privKeyEntry.config(state="normal")
    pubKeyEntry.config(state="normal")
    privKeyEntry.delete(0, END)
    pubKeyEntry.delete(0, END)
    privKeyEntry.insert(0, str(hex(keys[0]))[2:])
    pubKeyEntry.insert(0, keys[2])
    privKeyEntry.config(state="readonly")
    pubKeyEntry.config(state="readonly")
    t1.delete("1.0", END)
    t2.delete("1.0", END)
    privClave = keys[0]
    pubClave = keys[1]
    claveKompartida = cifradoClavePriv * pubClave
    claveKompartida = claveKompartida.x

def generate_keys():
    clavePriv = sc.randbelow(CURVA.field.n)
    clavePub = clavePriv * CURVA.g
    comClavePub = '0' + str(2 + clavePub.y % 2) + str(hex(clavePub.x)[2:])
    return clavePriv, clavePub, comClavePub

def generate_dynamic_privk():
    global cifradoClavePriv, claveKompartida
    cifradoClavePriv = sc.randbelow(CURVA.field.n)
    t1.delete("1.0", END)
    t2.delete("1.0", END)
    claveKompartida = cifradoClavePriv * pubClave
    claveKompartida = claveKompartida.x

def xor_simple(data, clave):
    return bytes(a ^ b for a, b in zip(data, clave))

def enkryptar(msg):
    global secreto
    claveKompartidaB = claveKompartida.to_bytes((claveKompartida.bit_length() + 7) // 8, 'big')
    claveKompartidaB = claveKompartidaB.ljust(len(msg), b'\0')
    ciphertxt = xor_simple(msg, claveKompartidaB)
    secreto = cifradoClavePriv * CURVA.g
    return ciphertxt

def desenkryptar(cMsg):
    claveKompartidaB = claveKompartida.to_bytes((claveKompartida.bit_length() + 7) // 8, 'big')
    claveKompartidaB = claveKompartidaB.ljust(len(cMsg), b'\0')
    plaintext = xor_simple(cMsg, claveKompartidaB)
    return plaintext

def procesar_text1(event=None):
    txt = t1.get("1.0", END).strip()
    txt_output = enkryptar(txt.encode("utf-8"))
    txt_output_b = base64.b64encode(txt_output)
    t2.delete("1.0", END)
    t2.insert("1.0", txt_output_b.decode("utf-8"))

def procesar_text2(event=None):
    txt = t2.get("1.0", END).strip()
    x1 = txt.encode("utf-8")
    valid_length = len(x1) - (len(x1) % 4)
    valid_base64_str = x1[:valid_length]
    try:
        x2 = base64.b64decode(valid_base64_str)
        txt_output = desenkryptar(x2)
        t1.delete("1.0", END)
        try:
            t1.insert("1.0", txt_output.decode("utf-8"))
        except UnicodeDecodeError as e:
            t1.insert("1.0", "Error: No se pudo decodificar el texto.")
    except binascii.Error as e:
        print("Error decodificando base64:", e)
        t1.delete("1.0", END)
        t1.insert("1.0", "Error: La cadena base64 es inválida.")

def boton_clave():
    global claveKompartida
    if clave_boton["text"] == "Añadir":
        clave = secretEntry.get().strip().encode("utf-8")
        if len(clave) < 24 or len(clave) > 32:
            secretEntry.delete(0, END)
            secretEntry.insert(0, "Error: La clave debe tener entre 24 y 32 caracteres.")
        else:
            clave_boton["text"] = "Eliminar"
            secretEntry.config(state=NORMAL)
            t1.delete("1.0", END)
            t2.delete("1.0", END)
            claveKompartida=int.from_bytes(clave, 'big')
    else:
        clave_boton["text"] = "Añadir"
        secretEntry.delete(0, END)
        secretEntry.config(state=NORMAL)
        t1.delete("1.0", END)
        t2.delete("1.0", END)
        claveKompartida = cifradoClavePriv * pubClave
        claveKompartida = claveKompartida.x


root = Tk()
root.title("Cifrado asimétrico")
root.resizable(False, False)

mainframe = ttk.Frame(root, padding="3 3 12 12")
keys_frame = ttk.Frame(root)

keys = generate_keys()
privClave = keys[0]
privKey_as_hex = str(hex(keys[0]))[2:]
pubClave = keys[1]
cifradoClavePriv = sc.randbelow(CURVA.field.n)
secreto = cifradoClavePriv * CURVA.g
claveKompartida = cifradoClavePriv * pubClave
claveKompartida = claveKompartida.x

mainframe.grid(column=0, row=1, sticky=(N, W, E, S))
keys_frame.grid(column=0, row=0, sticky=(N, W, E, S))

root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)

t1 = Text(mainframe, width=39, height=20, wrap="char")
t2 = Text(mainframe, width=39, height=20, wrap="char")

privKeylbl = ttk.Label(keys_frame, text="Clave privada:")
pubKeylbl = ttk.Label(keys_frame, text="Clave pública:")
secretlbl = ttk.Label(keys_frame, text="Ingresa una clave\n(24-32 caracteres):")

privKeyEntry = ttk.Entry(keys_frame, width=len(str(privKey_as_hex)))
pubKeyEntry = ttk.Entry(keys_frame, width=len(str(keys[2])))
secretEntry = ttk.Entry(keys_frame, width=35)

privKeyEntry.insert(0, privKey_as_hex)
privKeyEntry.config(state='readonly')

pubKeyEntry.insert(0, keys[2])
pubKeyEntry.config(state='readonly')

ttk.Button(keys_frame, text="Generar par", command=regenerate_keys).grid(column=2, row=0, sticky=W, padx=10, pady=(5, 0))
ttk.Button(keys_frame, text="Nueva clave dinámica", command=generate_dynamic_privk).grid(column=2, row=1, sticky=W, padx=10)

clave_boton = Button(keys_frame, text="Añadir", command=boton_clave)
clave_boton.grid(column=2, row=2, sticky=(S, W))

t1.grid(column=0, row=0, padx=(20, 0), pady=15, sticky=(E, S))
t2.grid(column=1, row=0, padx=20, pady=15, sticky=(W, S))

privKeylbl.grid(column=0, row=0, sticky=S, padx=20, pady=10)
pubKeylbl.grid(column=0, row=1, sticky=N, padx=20)
secretlbl.grid(column=0, row=2, sticky=W, padx=20, pady=(4,0), columnspan=2)

privKeyEntry.grid(column=1, row=0, sticky=W)
pubKeyEntry.grid(column=1, row=1, sticky=(N, W))
secretEntry.grid(column=1, row=2, sticky=(W, S), padx=(20, 0))

t1.bind("<KeyRelease>", procesar_text1)
t2.bind("<KeyRelease>", procesar_text2)

root.mainloop()